In [1]:
import re
import numpy as np
import pandas as pd

In [2]:
path_to_data = 'data/ECU_IoHT.xlsx'
data = pd.read_excel(path_to_data)

In [3]:
def extract_source_destination_port(text):
    match = re.search(r'(\d+)\s*>\s*(\d+)', text)
    if match:
        source, destination = match.groups()
        # Remove the matched substring from text
        updated_text = text.replace(match.group(0), '', 1).strip()
        return int(source), int(destination), updated_text
    else:
        return np.nan, np.nan, text

In [4]:
df = pd.DataFrame()
df[['source_p', 'destination_p', 'updated_info']] = data.Info.apply(lambda x: pd.Series(extract_source_destination_port(x)))

In [5]:
df['updated_info'].nunique(), data['Info'].nunique()

(8833, 24010)

In [6]:
def extract_seq(text):
    # Adjust regex to match the sequence number
    match = re.search(r'seq\s*=\s*(\d+)', text, re.IGNORECASE)  #
    if match:
        Seq = match.group(1)  # Get the first group directly
        updated_text = text.replace(match.group(0), '', 1).strip()
        return int(Seq), updated_text
    else:
        return np.nan, text


In [7]:
df2 = pd.DataFrame()
df2[['seq', 'updated_info']] = df['updated_info'].apply(lambda x: pd.Series(extract_seq(x)))

In [8]:
df2['updated_info'].nunique(), df['updated_info'].nunique(), data['Info'].nunique()

(7677, 8833, 24010)

In [9]:
df2[df2['seq'].notna()]['seq'].nunique()

2379

In [10]:
def extract_transaction_id(text):
    # Regex to match transaction ID starting with 0x and followed by hexadecimal characters
    match = re.search(r'0x[0-9a-fA-F]+', text)
    
    if match:
        # Extract the transaction ID as a string
        transaction_id = match.group(0)
        
        # Remove the transaction ID from the original text
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        
        # Return the integer value of the transaction ID and the updated text
        return int(transaction_id, 16), updated_text  # Convert hex to integer and return
        
    else:
        # If no transaction ID is found, return np.nan for both values
        return np.nan, text


In [11]:
df3 = pd.DataFrame()
df3[['transaction_id', 'updated_info']] = df2['updated_info'].apply(lambda x: pd.Series(extract_transaction_id(x)))

In [12]:
df3['updated_info'].nunique(), df2['updated_info'].nunique(), df['updated_info'].nunique(), data['Info'].nunique()

(7166, 7677, 8833, 24010)

In [29]:
def extract_len(text):
    match = re.search(r'Len=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [30]:
df4 = pd.DataFrame()
df4[['Len', 'updated_info']] = df3['updated_info'].apply(lambda x: pd.Series(extract_len(x)))

In [37]:
df4[df4['Len'].notna()]

,Len,updated_info
6,0.0,[SYN] Win=1024 MSS=1460
7,0.0,[SYN] Win=1024 MSS=1460
8,0.0,[SYN] Win=1024 MSS=1460
9,0.0,[SYN] Win=1024 MSS=1460
10,0.0,[SYN] Win=1024 MSS=1460
...,...,...
111197,69.0,"[TCP Retransmission] [PSH, ACK] Ack=1 Win=4565"
111198,0.0,[ACK] Ack=76 Win=4490
111199,341.0,"[TCP Retransmission] [PSH, ACK] Ack=76 Win=4490"
111200,0.0,[ACK] Ack=518 Win=4048


In [40]:
df4['updated_info'].nunique(), df3['updated_info'].nunique()

(7132, 7166)

In [16]:
df4['updated_info'].value_counts()

updated_info
Echo (ping) request  id=, /59262, ttl=64 (no response found!)                                             77715
[SYN]  Win=1024 Len=0 MSS=1460                                                                             7307
Application Data                                                                                           4029
[RST, ACK]  Ack=1 Win=5840 Len=0                                                                           3426
[RST, ACK]  Ack=1 Win=0 Len=0                                                                              2798
                                                                                                          ...  
[TCP Dup ACK 111124#1]  [ACK]  Ack=1 Win=4565 Len=0                                                           1
[TCP Dup ACK 111133#1]  [ACK]  Ack=76 Win=4490 Len=0                                                          1
[TCP Dup ACK 111157#1]  [ACK]  Ack=1 Win=4565 Len=0                                        

In [17]:
def extract_rquests_vs_responce():
    pass

In [44]:
def extract_win(text):
    match = re.search(r'Win=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [45]:
df5 = pd.DataFrame()
df5[['Win', 'updated_info']] = df4['updated_info'].apply(lambda x: pd.Series(extract_win(x)))

In [46]:
df5['updated_info'].nunique(),df4['updated_info'].nunique(), df3['updated_info'].nunique()

(7114, 7132, 7166)

In [48]:
df5[df5['Win'].notna()].value_counts()

Win       updated_info                                      
1024.0    [SYN]    MSS=1460                                     7307
5840.0    [RST, ACK]  Ack=1                                     3426
0.0       [RST, ACK]  Ack=1                                     2798
32767.0   [SYN]                                                 1023
5840.0    [TCP Retransmission]  [SYN]    MSS=1460                329
                                                                ... 
757376.0  [ACK]  Ack=740746   TSval=2005773 TSecr=1956108789       1
          [ACK]  Ack=698506   TSval=2005767 TSecr=1956108789       1
          [ACK]  Ack=701322   TSval=2005769 TSecr=1956108789       1
          [ACK]  Ack=704138   TSval=2005770 TSecr=1956108789       1
          [ACK]  Ack=706954   TSval=2005770 TSecr=1956108789       1
Name: count, Length: 6423, dtype: int64

In [49]:
def extract_MSS(text):
    match = re.search(r'MSS=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [54]:
df6 = pd.DataFrame()
df6[['MSS', 'updated_info']] = df5['updated_info'].apply(lambda x: pd.Series(extract_MSS(x)))

In [58]:
df6['updated_info'].nunique(),df5['updated_info'].nunique()

(7112, 7114)

In [63]:
df6[df6['MSS'].notna()].value_counts()

MSS     updated_info                                                          
1460.0  [SYN]                                                                     7362
        [TCP Retransmission]  [SYN]                                                329
        [SYN]     SACK_PERM=1 TSval=2012018 TSecr=0 WS=128                           6
1420.0  [SYN, ACK]  Ack=1    WS=512 SACK_PERM=1 TSval=1956203515 TSecr=2029432       5
1460.0  [SYN]     SACK_PERM=1 TSval=2028589 TSecr=0 WS=128                           5
                                                                                  ... 
        [SYN]     SACK_PERM=1 TSval=74461 TSecr=0 WS=128                             1
        [SYN]     SACK_PERM=1 TSval=75595 TSecr=0 WS=128                             1
        [SYN]    WS=1024  TSval=4294967295 TSecr=0 SACK_PERM=1                       1
        [TCP Retransmission]  [SYN, ACK]  Ack=1                                      1
265.0   [<None>]    WS=1024  TSval=4294967295 TSecr

In [50]:
def extract_Ack(text):
    match = re.search(r'Ack=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [55]:
df7 = pd.DataFrame()
df7[['Ack', 'updated_info']] = df6['updated_info'].apply(lambda x: pd.Series(extract_Ack(x)))

In [59]:
df7['updated_info'].nunique(),df6['updated_info'].nunique()

(5361, 7112)

In [ ]:
df7[df7['Ack'].notna()].value_counts()

Ack           updated_info                                                                                            
1.000000e+00  [RST, ACK]                                                                                                  6224
              [TCP Retransmission]  [PSH, ACK]                                                                             200
              [ACK]                                                                                                         86
5.190000e+02  [RST, ACK]                                                                                                    81
4.440000e+02  [RST, ACK]                                                                                                    69
                                                                                                                          ... 
4.213790e+08  [TCP Dup ACK 8358#1] [TCP ACKed unseen segment]  [ACK]     WS=1024  TSval=4294967295 TSecr=0 SACK_PERM=1 

In [51]:
def extract_TSval(text):
    match = re.search(r'TSval=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [56]:
df8 = pd.DataFrame()
df8[['TSval', 'updated_info']] = df7['updated_info'].apply(lambda x: pd.Series(extract_TSval(x)))

In [60]:
df8['updated_info'].nunique(),df7['updated_info'].nunique()

(4451, 5361)

In [65]:
df8[df8['TSval'].notna()].value_counts()

TSval         updated_info                                                                                
1.956108e+09  [ACK]      TSecr=2005604 [TCP segment of a reassembled PDU]                                     54
              [ACK]      TSecr=2005510 [TCP segment of a reassembled PDU]                                     48
              [ACK]      TSecr=2005428 [TCP segment of a reassembled PDU]                                     40
1.956198e+09  [ACK]      TSecr=2027972 [TCP segment of a reassembled PDU]                                     39
              [ACK]      TSecr=2027958 [TCP segment of a reassembled PDU]                                     28
                                                                                                              ..
4.294967e+09  [TCP Port numbers reused]  [SYN]    WS=1024   TSecr=0 SACK_PERM=1                                1
              [TCP Previous segment not captured]  [FIN, PSH, URG]   Urg=0  WS=16384   TSecr=0 SACK_PE

In [52]:
def extract_TSecr(text):
    match = re.search(r'TSecr=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [57]:
df9 = pd.DataFrame()
df9[['TSecr', 'updated_info']] = df8['updated_info'].apply(lambda x: pd.Series(extract_TSecr(x)))

In [61]:
df9['updated_info'].nunique(),df8['updated_info'].nunique()

(960, 4451)

In [66]:
df9[df9['TSecr'].notna()].value_counts()

TSecr         updated_info                                              
0.000000e+00  [SYN]     SACK_PERM=1   WS=128                                184
2.005604e+06  [ACK]       [TCP segment of a reassembled PDU]                 94
2.028086e+06  [ACK]       [TCP segment of a reassembled PDU]                 90
2.005510e+06  [ACK]       [TCP segment of a reassembled PDU]                 71
2.005428e+06  [ACK]       [TCP segment of a reassembled PDU]                 55
                                                                           ... 
0.000000e+00  [TCP Port numbers reused]  [SYN]    WS=1024    SACK_PERM=1      1
1.956206e+09  [ACK]                                                           1
              [ACK]                                                           1
              [ACK]                                                           1
              [ACK]                                                           1
Name: count, Length: 3617, dtype: int64

In [67]:
def extract_WS(text):
    match = re.search(r'WS=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [69]:
df10 = pd.DataFrame()
df10[['WS', 'updated_info']] = df9['updated_info'].apply(lambda x: pd.Series(extract_WS(x)))

In [71]:
df10['updated_info'].nunique(),df9['updated_info'].nunique()

(954, 960)

In [73]:
df10[df10['WS'].notna()].value_counts()

WS       updated_info                                                                             
128.0    [SYN]     SACK_PERM=1                                                                        184
512.0    [SYN, ACK]       SACK_PERM=1                                                                 174
256.0    [SYN, ACK]      SACK_PERM=1                                                                    7
32.0     [SYN, ACK]       SACK_PERM=1                                                                   7
1024.0   [ACK]         SACK_PERM=1                                                                      3
         [SYN]        SACK_PERM=1                                                                       3
         [TCP Retransmission]  [FIN, SYN, PSH, URG]   Urg=0      SACK_PERM=1                            3
16384.0  [FIN, PSH, URG]   Urg=0      SACK_PERM=1                                                       2
         [TCP Retransmission]  [FIN, PSH, URG]   Urg=

In [68]:
def extract_Urg(text):
    match = re.search(r'Urg=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [70]:
df11 = pd.DataFrame()
df11[['Urg', 'updated_info']] = df10['updated_info'].apply(lambda x: pd.Series(extract_Urg(x)))

In [72]:
df11['updated_info'].nunique(),df10['updated_info'].nunique()

(954, 954)

In [74]:
df11[df11['Urg'].notna()].value_counts()

Urg  updated_info                                                            
0.0  [TCP Retransmission]  [FIN, SYN, PSH, URG]         SACK_PERM=1              3
     [FIN, PSH, URG]         SACK_PERM=1                                         2
     [TCP Retransmission]  [FIN, PSH, URG]         SACK_PERM=1                   2
     [TCP Previous segment not captured]  [FIN, PSH, URG]         SACK_PERM=1    1
     [FIN, SYN, PSH, URG]         SACK_PERM=1                                    1
Name: count, dtype: int64

In [75]:
df11['updated_info'].value_counts()

updated_info
Echo (ping) request  id=, /59262, ttl=64 (no response found!)                                77715
[SYN]                                                                                         8386
[RST, ACK]                                                                                    6429
[ACK]                                                                                         5395
Application Data                                                                              4029
                                                                                             ...  
[TCP Previous segment not captured]  [FIN, PSH, URG]         SACK_PERM=1                         1
[TCP ACKed unseen segment] [TCP Previous segment not captured]  [ACK]         SACK_PERM=1        1
[TCP Port numbers reused]  [SYN]        SACK_PERM=1                                              1
Echo (ping) request  id=, /10241, ttl=45 (no response found!)                                   

In [77]:
def extract_SACK_PERM(text):
    match = re.search(r'SACK_PERM=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [78]:
df12 = pd.DataFrame()
df12[['SACK_PERM', 'updated_info']] = df11['updated_info'].apply(lambda x: pd.Series(extract_SACK_PERM(x)))

In [81]:
df12['updated_info'].nunique(),df11['updated_info'].nunique()

(946, 954)

In [83]:
df12['updated_info'].value_counts()

updated_info
Echo (ping) request  id=, /59262, ttl=64 (no response found!)    77715
[SYN]                                                             8577
[RST, ACK]                                                        6429
[ACK]                                                             5398
Application Data                                                  4029
                                                                 ...  
[TCP Dup ACK 111011#1]  [ACK]                                        1
[TCP Dup ACK 110999#2]  [ACK]                                        1
[TCP Dup ACK 110999#1]  [ACK]                                        1
[TCP Dup ACK 110955#1]  [ACK]                                        1
[TCP Dup ACK 110944#2]  [ACK]                                        1
Name: count, Length: 946, dtype: int64

In [79]:
def extract_ttl(text):
    match = re.search(r'ttl=(\d+)', text)
    if match:
        length = match.group(1) 
        updated_text = text.replace(match.group(0), '').strip()  # Strip to clean up any leading/trailing spaces
        return int(length), updated_text
    else:
        return np.nan, text

In [80]:
df13 = pd.DataFrame()
df13[['ttl', 'updated_info']] = df12['updated_info'].apply(lambda x: pd.Series(extract_ttl(x)))

In [82]:
df13['updated_info'].nunique(),df12['updated_info'].nunique()

(946, 946)

In [86]:
df13['updated_info'].value_counts().to_csv('info946.csv')

In [ ]:
def extract_seq_ack(text):
    # Regular expression to match the sequence number and acknowledgment number
    match = re.search(r'TCP Dup ACK (\d+)#(\d+)', text)
    
    if match:
        seq_num = match.group(1)  # Sequence number
        ack_num = match.group(2)  # Acknowledgment number
        updated_text = text.replace(match.group(0), '').strip()  # Clean up the text after extraction
        return int(seq_num), int(ack_num), updated_text
    else:
        return np.nan, np.nan, text

In [ ]:
df14 = pd.DataFrame()
df14[['seq_ack', 'ack_num', 'updated_info']] = df13['updated_info'].apply(lambda x: pd.Series(extract_seq_ack(x)))

In [90]:
df14['updated_info'].nunique(),df13['updated_info'].nunique()

(741, 946)

In [92]:
df14['updated_info'].value_counts().to_csv('info741.csv')

In [95]:
def extract_ips_from_parts(text):
    # Regular expression to match IPv4 addresses
    match = re.search(r'Who has (\b(?:\d{1,3}\.){3}\d{1,3}\b).*?Tell (\b(?:\d{1,3}\.){3}\d{1,3}\b)', text)
    
    if match:
        # Extract the "Who has" IP and "Tell" IP
        who_has_ip = match.group(1)
        tell_ip = match.group(2)
        
        # Clean up the text after extracting the IPs
        updated_text = text.replace(match.group(0), '').strip()
        
        return who_has_ip, tell_ip, updated_text
    else:
        # If no match found, return np.nan for both IPs and the original text
        return np.nan, np.nan, text


In [96]:
df15 = pd.DataFrame()
df15[['who_has_ip', 'tell_ip', 'updated_info']] = df14['updated_info'].apply(lambda x: pd.Series(extract_ips_from_parts(x)))

In [97]:
df15['updated_info'].nunique(),df14['updated_info'].nunique()

(486, 741)

In [99]:
df15['updated_info'].value_counts().to_csv('info486.csv')